In [1]:
import numpy as np
import math
import copy

gamma = 0.6
N_e = 4

In [2]:
class State:
    ball_x = 0
    ball_y = 0
    velocity_x = 0
    velocity_y = 0
    paddle_y = 0
    terminate = False
    
    
    def print_state(self):
        print(np.around(self.ball_x, 2) , " " , np.around(self.ball_y, 2) , " " ,  
              np.around(self.velocity_x, 2) , " " ,  np.around(self.velocity_y, 2) , " " ,  np.around(self.paddle_y, 2))
              
 
    def __init__(self, bx, by, vx, vy, p):
        self.ball_x = bx
        self.ball_y = by
        self.velocity_x = vx
        self.velocity_y = vy
        self.paddle_y = p
        
         
    def up(self):
        self.paddle_y = max(self.paddle_y - 0.04, 0)
        
        self.update()
        #print("in function up", match(self))

    def down(self):
        self.paddle_y = min(self.paddle_y + 0.04, 0.8)
        self.update()

    def update(self):
        global num_bounce
#         global state_tracker
        
        self.ball_x += self.velocity_x
        self.ball_y += self.velocity_y
        
        if self.ball_x >= 1:        # hitting right bound
            
            
            if self.ball_y >= self.paddle_y and self.ball_y <= self.paddle_y + 0.2:    # hitting paddle
                self.ball_x = 2 - self.ball_x
                #print("bounce")
                num_bounce += 1
#                 if curr_state not in state_tracker:
#                     #print(curr_state)
#                     state_tracker.append(curr_state)
                                
                self.velocity_x = -self.velocity_x + np.random.uniform(-0.015, 0.015)
                if self.velocity_x < 0:
                    self.velocity_x = min(-0.03, self.velocity_x)
                else:
                    self.velocity_x = max(0.03, self.velocity_x)
                if self.velocity_x > 1:
                    self.velocity_x = 1
                elif self.velocity_x < -1:
                    self.velocity_x = -1
                
                self.velocity_y += np.random.uniform(-0.03, 0.03)
                if self.velocity_y > 1:
                    self.velocity_y = 1
                elif self.velocity_y < -1:
                    self.velocity_y = -1
                    
                
                state_tracker = []
                
            else:    # missing paddle
                #print("terminate")
                self.ball_x = 2 - self.ball_x
#                 reward(state_tracker,-1)
#                 state_tracker = []
                self.terminate = True

            
        if self.ball_x <= 0:     # hitting left bound
            self.ball_x = -self.ball_x
            self.velocity_x = -self.velocity_x
        if self.ball_y <= 0:     # hitting upper bound
            self.ball_y = -self.ball_y
            self.velocity_y = -self.velocity_y
        if self.ball_y  >= 1:    # hitting right bound
            self.ball_y = 2 - self.ball_y
            self.velocity_y = -self.velocity_y

In [3]:
def match(s):
    b_x = math.floor(s.ball_x * 12)
    b_y = math.floor(s.ball_y * 12)
    if s.velocity_x > 0:
        v_x = 1
    else:
        v_x = 0    # velocity_x <= 0 map to 0

    v_y = 0
    if s.velocity_y > 0 and abs(s.velocity_y) > 0.015:
        v_y = 1
    elif s.velocity_y < 0 and abs(s.velocity_y) > 0.015:    #velocity_y < 0.015 map to 2
        v_y = 2

    #print("paddle_y", s.paddle_y)
    p = math.floor(s.paddle_y / (1 - 0.2) * 12)
    if p == 12:
        p = 11
    #print("p", p)

    return (b_x ,b_y, v_x, v_y, p)

In [4]:
def reward(state):
    next_ball_x = state.ball_x + state.velocity_x

    if next_ball_x >= 1:        # hitting right bound
        if state.ball_y >= state.paddle_y and state.ball_y <= state.paddle_y + 0.2:    # hitting paddle
            return 1
        else:     # missing paddle
            return -1
    return 0

In [5]:

def get_up(current_state):
    #print("up current", match(current_state))
    new_state = State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.up()
    #print("new state", match(new_state))
    return new_state

def get_down(current_state):
    new_state = State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.down()
    return new_state

def get_no_move(current_state):
    new_state = State(current_state.ball_x, current_state.ball_y, current_state.velocity_x, \
                      current_state.velocity_y, current_state.paddle_y)
    new_state.update()
    return new_state

In [6]:
def max_Q(current_state):
    (b_x ,b_y, v_x, v_y, p) = match(get_up(current_state))
    Q_up = Q_table[b_x][b_y][v_x][v_y][p][0]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_down(current_state))
    #print(b_x ,b_y, v_x, v_y, p)
    Q_down = Q_table[b_x][b_y][v_x][v_y][p][1]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_no_move(current_state))
    Q_no_move = Q_table[b_x][b_y][v_x][v_y][p][2]
    
    #print(Q_up, Q_down, Q_no_move)
    
    return max(Q_up, Q_down, Q_no_move)

In [7]:
def argmax_f(current_state):
    (b_x ,b_y, v_x, v_y, p) = match(get_up(current_state))
    if N_sa[b_x][b_y][v_x][v_y][p][0] < N_e:
        f_up = 0
    else:
        f_up = Q_table[b_x][b_y][v_x][v_y][p][0]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_down(current_state))
    if N_sa[b_x][b_y][v_x][v_y][p][1] < N_e:
        f_down = 0
    else:
        f_down = Q_table[b_x][b_y][v_x][v_y][p][1]
        
    (b_x ,b_y, v_x, v_y, p) = match(get_no_move(current_state))
    if N_sa[b_x][b_y][v_x][v_y][p][2] < N_e:
        f_no_move = 0
    else:
        f_no_move = Q_table[b_x][b_y][v_x][v_y][p][2]
    
    if f_up == f_down and f_down == f_no_move:
        return np.random.randint(3)
    
    return np.argmax([f_up, f_down, f_no_move])

In [8]:
def alpha(state):
    (b_x ,b_y, v_x, v_y, p) = match(state)
    num_visited = N_sa[b_x][b_y][v_x][v_y][p][a] 
    return 60 / (60 + num_visited)

In [9]:
Q_table = np.zeros((12, 12, 2, 3, 12, 3)) # x, y, v_x, v_y, paddle, action
Q_additional_state = -1                   # x >= 1 state
N_sa = np.zeros((12, 12, 2, 3, 12, 3))    # x, y, v_x, v_y, paddle, action

s = State(0,0,0,0,0)
s_empty = True
a = 0
r = 0

In [10]:
'''action: 0-up, 1-down, 2-no_move'''
def Q_agent(current_state, t):
    global s_empty
    global s
    global a
    global r
    global N_sa
    global Q
    if current_state.terminate:
        (b_x ,b_y, v_x, v_y, p) = match(s)
        #Q_table[b_x][b_y][v_x][v_y][p][3] = reward(current_state)

    if s_empty is False:
        # increment N_s
        (b_x ,b_y, v_x, v_y, p) = match(s)
        #print(b_x ,b_y, v_x, v_y, p, a)
        N_sa[b_x][b_y][v_x][v_y][p][a] += 1
        #print(N_sa[b_x][b_y][v_x][v_y][p][a])
        
        Q_table[b_x][b_y][v_x][v_y][p][a] =  Q_table[b_x][b_y][v_x][v_y][p][a] + \
                            alpha(s)* (r + gamma * max_Q(current_state) - Q_table[b_x][b_y][v_x][v_y][p][a])
        
    else:
        s_empty = False
    
    #s.empty = False
    s = copy.deepcopy(current_state)
    #print("s", match(s))
    
    (b_x ,b_y, v_x, v_y, p) = match(current_state)
    a = argmax_f(current_state)
    
    r = reward(current_state)
    
    return a

In [11]:

t_ = 1
num_bounce = 0
total_num_bounce = 0
for i in range(10000):
    s = State(0.5, 0.5, 0.03, 0.01, 0.4)
    current_state = copy.deepcopy(s)
    print("round: ", i)
    num_bounce = 0
    while s.terminate is False:
        #s.print_state()
        next_action = Q_agent(current_state, t_)
        #print("current state before action", match(current_state))
        #print("next action", next_action)
        if next_action == 0:
            current_state.up()
        if next_action == 1:
            current_state.down()
        if next_action == 2:
            current_state.update()
        #print("current state", match(current_state))
    t_ += 1
    print("bounce:", num_bounce)
    total_num_bounce += num_bounce
    print("state visited: ",np.count_nonzero(Q_table))
# print(total_num_bounce / 10000)

round:  0
bounce: 0
state visited:  1
round:  1
bounce: 7
state visited:  4
round:  2
bounce: 9
state visited:  8
round:  3
bounce: 0
state visited:  10
round:  4
bounce: 0
state visited:  12
round:  5
bounce: 0
state visited:  15
round:  6
bounce: 0
state visited:  17
round:  7
bounce: 2
state visited:  21
round:  8
bounce: 9
state visited:  25
round:  9
bounce: 0
state visited:  31
round:  10
bounce: 0
state visited:  35
round:  11
bounce: 0
state visited:  38
round:  12
bounce: 0
state visited:  41
round:  13
bounce: 0
state visited:  41
round:  14
bounce: 0
state visited:  44
round:  15
bounce: 0
state visited:  47
round:  16
bounce: 0
state visited:  48
round:  17
bounce: 0
state visited:  50
round:  18
bounce: 0
state visited:  53
round:  19
bounce: 0
state visited:  55
round:  20
bounce: 0
state visited:  55
round:  21
bounce: 0
state visited:  56
round:  22
bounce: 0
state visited:  56
round:  23
bounce: 0
state visited:  56
round:  24
bounce: 0
state visited:  56
round:  25
bo

bounce: 12
state visited:  288
round:  224
bounce: 7
state visited:  289
round:  225
bounce: 7
state visited:  291
round:  226
bounce: 7
state visited:  292
round:  227
bounce: 5
state visited:  294
round:  228
bounce: 5
state visited:  295
round:  229
bounce: 7
state visited:  296
round:  230
bounce: 7
state visited:  297
round:  231
bounce: 7
state visited:  298
round:  232
bounce: 5
state visited:  301
round:  233
bounce: 14
state visited:  305
round:  234
bounce: 5
state visited:  305
round:  235
bounce: 5
state visited:  306
round:  236
bounce: 7
state visited:  308
round:  237
bounce: 5
state visited:  309
round:  238
bounce: 7
state visited:  309
round:  239
bounce: 7
state visited:  310
round:  240
bounce: 7
state visited:  311
round:  241
bounce: 7
state visited:  311
round:  242
bounce: 5
state visited:  312
round:  243
bounce: 4
state visited:  312
round:  244
bounce: 8
state visited:  315
round:  245
bounce: 7
state visited:  316
round:  246
bounce: 31
state visited:  326
r

round:  424
bounce: 5
state visited:  703
round:  425
bounce: 5
state visited:  703
round:  426
bounce: 7
state visited:  703
round:  427
bounce: 5
state visited:  704
round:  428
bounce: 17
state visited:  709
round:  429
bounce: 7
state visited:  709
round:  430
bounce: 12
state visited:  714
round:  431
bounce: 5
state visited:  715
round:  432
bounce: 23
state visited:  720
round:  433
bounce: 14
state visited:  724
round:  434
bounce: 10
state visited:  725
round:  435
bounce: 5
state visited:  726
round:  436
bounce: 10
state visited:  729
round:  437
bounce: 10
state visited:  734
round:  438
bounce: 5
state visited:  734
round:  439
bounce: 14
state visited:  737
round:  440
bounce: 5
state visited:  739
round:  441
bounce: 17
state visited:  742
round:  442
bounce: 7
state visited:  743
round:  443
bounce: 7
state visited:  749
round:  444
bounce: 7
state visited:  751
round:  445
bounce: 7
state visited:  753
round:  446
bounce: 5
state visited:  754
round:  447
bounce: 7
sta

bounce: 11
state visited:  1418
round:  628
bounce: 10
state visited:  1431
round:  629
bounce: 7
state visited:  1432
round:  630
bounce: 7
state visited:  1433
round:  631
bounce: 5
state visited:  1434
round:  632
bounce: 5
state visited:  1435
round:  633
bounce: 7
state visited:  1437
round:  634
bounce: 5
state visited:  1438
round:  635
bounce: 7
state visited:  1446
round:  636
bounce: 20
state visited:  1461
round:  637
bounce: 5
state visited:  1473
round:  638
bounce: 17
state visited:  1484
round:  639
bounce: 7
state visited:  1485
round:  640
bounce: 12
state visited:  1489
round:  641
bounce: 7
state visited:  1490
round:  642
bounce: 8
state visited:  1494
round:  643
bounce: 12
state visited:  1497
round:  644
bounce: 7
state visited:  1498
round:  645
bounce: 5
state visited:  1499
round:  646
bounce: 7
state visited:  1500
round:  647
bounce: 14
state visited:  1511
round:  648
bounce: 10
state visited:  1517
round:  649
bounce: 7
state visited:  1520
round:  650
bou

bounce: 7
state visited:  2447
round:  823
bounce: 21
state visited:  2454
round:  824
bounce: 7
state visited:  2455
round:  825
bounce: 14
state visited:  2467
round:  826
bounce: 12
state visited:  2474
round:  827
bounce: 5
state visited:  2476
round:  828
bounce: 19
state visited:  2487
round:  829
bounce: 5
state visited:  2487
round:  830
bounce: 9
state visited:  2494
round:  831
bounce: 7
state visited:  2500
round:  832
bounce: 21
state visited:  2505
round:  833
bounce: 5
state visited:  2506
round:  834
bounce: 10
state visited:  2518
round:  835
bounce: 7
state visited:  2519
round:  836
bounce: 7
state visited:  2520
round:  837
bounce: 7
state visited:  2522
round:  838
bounce: 5
state visited:  2525
round:  839
bounce: 5
state visited:  2530
round:  840
bounce: 33
state visited:  2549
round:  841
bounce: 5
state visited:  2554
round:  842
bounce: 5
state visited:  2558
round:  843
bounce: 12
state visited:  2562
round:  844
bounce: 14
state visited:  2565
round:  845
bo

...

bounce: 40
state visited:  18067
round:  9118
bounce: 52
state visited:  18069
round:  9119
bounce: 12
state visited:  18072
round:  9120
bounce: 60
state visited:  18072
round:  9121
bounce: 26
state visited:  18072
round:  9122
bounce: 7
state visited:  18072
round:  9123
bounce: 68
state visited:  18072
round:  9124
bounce: 59
state visited:  18072
round:  9125
bounce: 31
state visited:  18072
round:  9126
bounce: 28
state visited:  18072
round:  9127
bounce: 12
state visited:  18072
round:  9128
bounce: 17
state visited:  18072
round:  9129
bounce: 33
state visited:  18074
round:  9130
bounce: 11
state visited:  18074
round:  9131
bounce: 14
state visited:  18074
round:  9132
bounce: 9
state visited:  18074
round:  9133
bounce: 18
state visited:  18074
round:  9134
bounce: 31
state visited:  18075
round:  9135
bounce: 38
state visited:  18075
round:  9136
bounce: 28
state visited:  18075
round:  9137
bounce: 60
state visited:  18075
round:  9138
bounce: 21
state visited:  18080
rou

bounce: 47
state visited:  18170
round:  9305
bounce: 29
state visited:  18170
round:  9306
bounce: 16
state visited:  18170
round:  9307
bounce: 49
state visited:  18170
round:  9308
bounce: 48
state visited:  18170
round:  9309
bounce: 18
state visited:  18171
round:  9310
bounce: 11
state visited:  18171
round:  9311
bounce: 35
state visited:  18171
round:  9312
bounce: 12
state visited:  18171
round:  9313
bounce: 14
state visited:  18171
round:  9314
bounce: 59
state visited:  18171
round:  9315
bounce: 40
state visited:  18171
round:  9316
bounce: 21
state visited:  18171
round:  9317
bounce: 26
state visited:  18171
round:  9318
bounce: 31
state visited:  18173
round:  9319
bounce: 59
state visited:  18173
round:  9320
bounce: 30
state visited:  18173
round:  9321
bounce: 30
state visited:  18175
round:  9322
bounce: 45
state visited:  18176
round:  9323
bounce: 39
state visited:  18180
round:  9324
bounce: 54
state visited:  18182
round:  9325
bounce: 23
state visited:  18182
r

bounce: 92
state visited:  18256
round:  9488
bounce: 20
state visited:  18256
round:  9489
bounce: 8
state visited:  18256
round:  9490
bounce: 20
state visited:  18256
round:  9491
bounce: 25
state visited:  18256
round:  9492
bounce: 43
state visited:  18256
round:  9493
bounce: 45
state visited:  18256
round:  9494
bounce: 21
state visited:  18257
round:  9495
bounce: 12
state visited:  18257
round:  9496
bounce: 14
state visited:  18257
round:  9497
bounce: 13
state visited:  18257
round:  9498
bounce: 69
state visited:  18257
round:  9499
bounce: 35
state visited:  18257
round:  9500
bounce: 59
state visited:  18258
round:  9501
bounce: 40
state visited:  18258
round:  9502
bounce: 59
state visited:  18261
round:  9503
bounce: 10
state visited:  18261
round:  9504
bounce: 16
state visited:  18261
round:  9505
bounce: 7
state visited:  18261
round:  9506
bounce: 21
state visited:  18261
round:  9507
bounce: 25
state visited:  18261
round:  9508
bounce: 14
state visited:  18261
rou

bounce: 24
state visited:  18376
round:  9673
bounce: 51
state visited:  18376
round:  9674
bounce: 9
state visited:  18376
round:  9675
bounce: 20
state visited:  18377
round:  9676
bounce: 12
state visited:  18377
round:  9677
bounce: 31
state visited:  18377
round:  9678
bounce: 15
state visited:  18377
round:  9679
bounce: 61
state visited:  18377
round:  9680
bounce: 45
state visited:  18377
round:  9681
bounce: 54
state visited:  18377
round:  9682
bounce: 9
state visited:  18377
round:  9683
bounce: 55
state visited:  18378
round:  9684
bounce: 19
state visited:  18378
round:  9685
bounce: 28
state visited:  18378
round:  9686
bounce: 10
state visited:  18378
round:  9687
bounce: 24
state visited:  18378
round:  9688
bounce: 12
state visited:  18380
round:  9689
bounce: 12
state visited:  18380
round:  9690
bounce: 16
state visited:  18380
round:  9691
bounce: 16
state visited:  18380
round:  9692
bounce: 18
state visited:  18380
round:  9693
bounce: 14
state visited:  18380
rou

bounce: 38
state visited:  18428
round:  9857
bounce: 32
state visited:  18428
round:  9858
bounce: 72
state visited:  18428
round:  9859
bounce: 11
state visited:  18428
round:  9860
bounce: 9
state visited:  18428
round:  9861
bounce: 10
state visited:  18428
round:  9862
bounce: 25
state visited:  18428
round:  9863
bounce: 13
state visited:  18428
round:  9864
bounce: 21
state visited:  18428
round:  9865
bounce: 11
state visited:  18428
round:  9866
bounce: 15
state visited:  18428
round:  9867
bounce: 36
state visited:  18428
round:  9868
bounce: 40
state visited:  18428
round:  9869
bounce: 15
state visited:  18429
round:  9870
bounce: 55
state visited:  18430
round:  9871
bounce: 31
state visited:  18431
round:  9872
bounce: 31
state visited:  18431
round:  9873
bounce: 114
state visited:  18432
round:  9874
bounce: 21
state visited:  18432
round:  9875
bounce: 17
state visited:  18436
round:  9876
bounce: 25
state visited:  18436
round:  9877
bounce: 31
state visited:  18436
r

In [12]:
print("average bouncing per game: ", total_num_bounce / 10000)

print("totoal state visited after training: ", np.count_nonzero(Q_table))



average bouncing per game:  23.1031
totoal state visited after training:  18506


In [13]:
print (sorted(Q_table[np.nonzero(Q_table)],reverse=True), '\n')

[0.9994767089677451, 0.985488298083119, 0.97658342562922629, 0.97631008101863459, 0.97079408567914871, 0.96775114772735771, 0.90901645204102932, 0.90428784805785589, 0.88394391041509546, 0.87655847153106803, 0.8472833485511605, 0.84525577647046413, 0.79952822771165521, 0.77097788318194749, 0.75338993035885393, 0.74418414224905816, 0.73870054008469743, 0.71775371000327803, 0.7146889196665589, 0.71444772711824811, 0.71196743999770529, 0.69806417911206498, 0.69401585164317059, 0.69358765743188899, 0.66926025654179688, 0.66720109986213638, 0.66603970002190183, 0.6587013037662911, 0.65703882658986079, 0.63792167955803869, 0.6368740208662339, 0.63371433669984034, 0.63119495714906626, 0.62314379570172729, 0.61977515164641495, 0.61751209150959752, 0.61596868120803672, 0.61363545536372444, 0.61211779542306166, 0.60832017705085351, 0.60803307622664859, 0.60471316761024618, 0.59912299255071222, 0.59041105088546952, 0.58956201156356647, 0.5855194041929902, 0.58463365204537288, 0.58187771498352348,

In [14]:
print (sorted(N_sa[np.nonzero(N_sa)],reverse=True), '\n')

[14274.0, 13918.0, 13903.0, 12639.0, 11422.0, 11391.0, 9027.0, 8703.0, 7702.0, 7618.0, 7270.0, 7030.0, 6597.0, 5999.0, 5618.0, 5497.0, 4168.0, 4097.0, 4089.0, 4019.0, 3361.0, 3257.0, 3115.0, 3052.0, 3047.0, 3036.0, 2947.0, 2931.0, 2801.0, 2778.0, 2767.0, 2758.0, 2725.0, 2706.0, 2642.0, 2635.0, 2635.0, 2561.0, 2506.0, 2480.0, 2470.0, 2402.0, 2400.0, 2374.0, 2314.0, 2267.0, 2257.0, 2233.0, 2171.0, 2137.0, 2102.0, 2045.0, 2039.0, 2020.0, 2020.0, 2003.0, 2001.0, 1994.0, 1976.0, 1943.0, 1940.0, 1922.0, 1880.0, 1816.0, 1812.0, 1806.0, 1796.0, 1794.0, 1793.0, 1715.0, 1709.0, 1701.0, 1698.0, 1694.0, 1672.0, 1671.0, 1650.0, 1623.0, 1619.0, 1607.0, 1595.0, 1588.0, 1571.0, 1557.0, 1537.0, 1534.0, 1527.0, 1526.0, 1515.0, 1514.0, 1494.0, 1487.0, 1471.0, 1452.0, 1432.0, 1426.0, 1423.0, 1422.0, 1416.0, 1414.0, 1412.0, 1410.0, 1407.0, 1403.0, 1389.0, 1385.0, 1385.0, 1381.0, 1379.0, 1374.0, 1372.0, 1360.0, 1358.0, 1335.0, 1332.0, 1332.0, 1331.0, 1327.0, 1322.0, 1320.0, 1316.0, 1308.0, 1307.0, 1293.0, 1

In [15]:
# print (N_sa[np.nonzero(N_sa)])

import statistics as stat

In [16]:
data = N_sa[np.nonzero(N_sa)]

In [17]:
print (stat.mean(data), stat.median(data), stat.mode(data))

print (stat.median_low(data), stat.median_high(data))

print (stat.median_grouped(data))

print (stat.harmonic_mean(data))

79.0241395065 8.0 1.0
8.0 8.0
8.05450236967
3.98691202075


In [18]:
print(stat.stdev(data))

308.3137486032565


In [21]:

t_ = 1
num_bounce = 0
total_num_bounce = 0
for i in range(1000):
    s = State(0.5, 0.5, 0.03, 0.01, 0.4)
    current_state = copy.deepcopy(s)
    print("round: ", i)
    num_bounce = 0
    while s.terminate is False:
        #s.print_state()
        next_action = Q_agent_test(current_state, t_)
        #print("current state before action", match(current_state))
        #print("next action", next_action)
        if next_action == 0:
            current_state.up()
        if next_action == 1:
            current_state.down()
        if next_action == 2:
            current_state.update()
        #print("current state", match(current_state))
    t_ += 1
    print("bounce:", num_bounce)
    total_num_bounce += num_bounce
    print("state visited: ",np.count_nonzero(Q_table))


round:  0
bounce: 10
state visited:  18506
round:  1
bounce: 14
state visited:  18506
round:  2
bounce: 19
state visited:  18506
round:  3
bounce: 6
state visited:  18506
round:  4
bounce: 5
state visited:  18506
round:  5
bounce: 21
state visited:  18506
round:  6
bounce: 12
state visited:  18506
round:  7
bounce: 21
state visited:  18506
round:  8
bounce: 19
state visited:  18506
round:  9
bounce: 7
state visited:  18506
round:  10
bounce: 7
state visited:  18506
round:  11
bounce: 10
state visited:  18506
round:  12
bounce: 16
state visited:  18506
round:  13
bounce: 7
state visited:  18506
round:  14
bounce: 26
state visited:  18506
round:  15
bounce: 8
state visited:  18506
round:  16
bounce: 8
state visited:  18506
round:  17
bounce: 7
state visited:  18506
round:  18
bounce: 15
state visited:  18506
round:  19
bounce: 55
state visited:  18506
round:  20
bounce: 18
state visited:  18506
round:  21
bounce: 15
state visited:  18506
round:  22
bounce: 39
state visited:  18506
round:

bounce: 13
state visited:  18506
round:  197
bounce: 6
state visited:  18506
round:  198
bounce: 18
state visited:  18506
round:  199
bounce: 22
state visited:  18506
round:  200
bounce: 39
state visited:  18506
round:  201
bounce: 52
state visited:  18506
round:  202
bounce: 10
state visited:  18506
round:  203
bounce: 10
state visited:  18506
round:  204
bounce: 18
state visited:  18506
round:  205
bounce: 6
state visited:  18506
round:  206
bounce: 15
state visited:  18506
round:  207
bounce: 43
state visited:  18506
round:  208
bounce: 8
state visited:  18506
round:  209
bounce: 10
state visited:  18506
round:  210
bounce: 60
state visited:  18506
round:  211
bounce: 30
state visited:  18506
round:  212
bounce: 11
state visited:  18506
round:  213
bounce: 12
state visited:  18506
round:  214
bounce: 23
state visited:  18506
round:  215
bounce: 20
state visited:  18506
round:  216
bounce: 6
state visited:  18506
round:  217
bounce: 11
state visited:  18506
round:  218
bounce: 26
sta

bounce: 18
state visited:  18506
round:  393
bounce: 9
state visited:  18506
round:  394
bounce: 6
state visited:  18506
round:  395
bounce: 8
state visited:  18506
round:  396
bounce: 21
state visited:  18506
round:  397
bounce: 7
state visited:  18506
round:  398
bounce: 6
state visited:  18506
round:  399
bounce: 7
state visited:  18506
round:  400
bounce: 9
state visited:  18506
round:  401
bounce: 39
state visited:  18506
round:  402
bounce: 8
state visited:  18506
round:  403
bounce: 11
state visited:  18506
round:  404
bounce: 12
state visited:  18506
round:  405
bounce: 18
state visited:  18506
round:  406
bounce: 31
state visited:  18506
round:  407
bounce: 20
state visited:  18506
round:  408
bounce: 23
state visited:  18506
round:  409
bounce: 8
state visited:  18506
round:  410
bounce: 5
state visited:  18506
round:  411
bounce: 41
state visited:  18506
round:  412
bounce: 26
state visited:  18506
round:  413
bounce: 23
state visited:  18506
round:  414
bounce: 19
state vis

bounce: 37
state visited:  18506
round:  578
bounce: 21
state visited:  18506
round:  579
bounce: 30
state visited:  18506
round:  580
bounce: 14
state visited:  18506
round:  581
bounce: 25
state visited:  18506
round:  582
bounce: 31
state visited:  18506
round:  583
bounce: 18
state visited:  18506
round:  584
bounce: 6
state visited:  18506
round:  585
bounce: 19
state visited:  18506
round:  586
bounce: 23
state visited:  18506
round:  587
bounce: 17
state visited:  18506
round:  588
bounce: 13
state visited:  18506
round:  589
bounce: 14
state visited:  18506
round:  590
bounce: 20
state visited:  18506
round:  591
bounce: 12
state visited:  18506
round:  592
bounce: 20
state visited:  18506
round:  593
bounce: 22
state visited:  18506
round:  594
bounce: 34
state visited:  18506
round:  595
bounce: 14
state visited:  18506
round:  596
bounce: 11
state visited:  18506
round:  597
bounce: 7
state visited:  18506
round:  598
bounce: 41
state visited:  18506
round:  599
bounce: 6
st

bounce: 11
state visited:  18506
round:  767
bounce: 6
state visited:  18506
round:  768
bounce: 20
state visited:  18506
round:  769
bounce: 26
state visited:  18506
round:  770
bounce: 8
state visited:  18506
round:  771
bounce: 6
state visited:  18506
round:  772
bounce: 10
state visited:  18506
round:  773
bounce: 18
state visited:  18506
round:  774
bounce: 7
state visited:  18506
round:  775
bounce: 14
state visited:  18506
round:  776
bounce: 9
state visited:  18506
round:  777
bounce: 18
state visited:  18506
round:  778
bounce: 16
state visited:  18506
round:  779
bounce: 11
state visited:  18506
round:  780
bounce: 12
state visited:  18506
round:  781
bounce: 8
state visited:  18506
round:  782
bounce: 15
state visited:  18506
round:  783
bounce: 15
state visited:  18506
round:  784
bounce: 8
state visited:  18506
round:  785
bounce: 13
state visited:  18506
round:  786
bounce: 12
state visited:  18506
round:  787
bounce: 11
state visited:  18506
round:  788
bounce: 12
state 

bounce: 39
state visited:  18506
round:  958
bounce: 6
state visited:  18506
round:  959
bounce: 11
state visited:  18506
round:  960
bounce: 16
state visited:  18506
round:  961
bounce: 22
state visited:  18506
round:  962
bounce: 23
state visited:  18506
round:  963
bounce: 10
state visited:  18506
round:  964
bounce: 23
state visited:  18506
round:  965
bounce: 27
state visited:  18506
round:  966
bounce: 12
state visited:  18506
round:  967
bounce: 48
state visited:  18506
round:  968
bounce: 29
state visited:  18506
round:  969
bounce: 5
state visited:  18506
round:  970
bounce: 26
state visited:  18506
round:  971
bounce: 8
state visited:  18506
round:  972
bounce: 6
state visited:  18506
round:  973
bounce: 20
state visited:  18506
round:  974
bounce: 9
state visited:  18506
round:  975
bounce: 12
state visited:  18506
round:  976
bounce: 20
state visited:  18506
round:  977
bounce: 15
state visited:  18506
round:  978
bounce: 45
state visited:  18506
round:  979
bounce: 14
stat

In [19]:
def Q_agent_test(current_state, t):
    global s_empty
    global s
    global a
    global r
    global N_sa
    global Q
    if current_state.terminate:
        (b_x ,b_y, v_x, v_y, p) = match(s)
        #Q_table[b_x][b_y][v_x][v_y][p][3] = reward(current_state)

    if s_empty is False:
        # increment N_s
        (b_x ,b_y, v_x, v_y, p) = match(s)
        
    else:
        s_empty = False
    
    #s.empty = False
    s = copy.deepcopy(current_state)
    #print("s", match(s))
    
    (b_x ,b_y, v_x, v_y, p) = match(current_state)
    a = argmax_f_test(current_state)
    
    r = reward(current_state)
    
    return a

In [20]:
def argmax_f_test(current_state):
    (b_x ,b_y, v_x, v_y, p) = match(get_up(current_state))
    f_up = Q_table[b_x][b_y][v_x][v_y][p][0]
    
    (b_x ,b_y, v_x, v_y, p) = match(get_down(current_state))
    f_down = Q_table[b_x][b_y][v_x][v_y][p][1]
        
    (b_x ,b_y, v_x, v_y, p) = match(get_no_move(current_state))
    f_no_move = Q_table[b_x][b_y][v_x][v_y][p][2]
    
    if f_up == f_down and f_down == f_no_move:
        return np.random.randint(3)
    
    return np.argmax([f_up, f_down, f_no_move])

In [22]:
print("average bouncing per game: ", total_num_bounce / 1000)

print("totoal state visited after training: ", np.count_nonzero(Q_table))

average bouncing per game:  16.171
totoal state visited after training:  18506
